In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

from inversion_utils import *
import pickle
from sklearn.model_selection import train_test_split

In [3]:
SEED = 0

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

torch.backends.cudnn.benchmark = True 
torch.backends.cuda.matmul.allow_tf32 = True

LLM = namedtuple('LLM', ['language_model', 'tokenizer', 'processor', 'name', 'model_type'])

In [4]:
model_type = 'llama'
# model_type = 'qwen'

# MODEL_VERSION = '3'
MODEL_VERSION = '3.1'
# MODEL_VERSION = '3.3'

MODEL_SIZE = '8B'
# MODEL_SIZE = '70B'

llm = select_llm(model_type, MODEL_VERSION=MODEL_VERSION, MODEL_SIZE=MODEL_SIZE)

Loading meta-llama/Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Simple generations

In [6]:
coef = 0.4
# [.4, .41, .42, .43, .44, .45] for 70b
max_tokens = 200

prompts_1 = ["There is a stray dog in front of my house, what should I do?",]

In [ ]:
concept_1 = "angry"
controller_1 = load_controller(llm, concept_1, path="../directions_moods_plus_llama70/")

In [7]:
test_concept_vector(controller_1, 
                    concept=concept_1, 
                    prompts=prompts_1, 
                    coef=coef, 
                    max_tokens=max_tokens, 
                    orig=True,)


========================== No Control ==========================


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That can be a concerning situation. Here are some steps you can take to help the stray dog and ensure your safety:

1. **Check for identification**: Look for a collar or tag with the dog's owner's contact information. If you find any identification, try to contact the owner to let them know where their dog is.
2. **Observe from a safe distance**: Keep a safe distance from the dog to avoid any potential aggression. Observe the dog's behavior, size, breed, and overall condition.
3. **Contact local animal control or a shelter**: Reach out to your local animal control agency or a nearby animal shelter to report the stray dog. They can send someone to pick up the dog and try to locate its owner.
4. **Pr

Training KRR

In [8]:
with open("../data/moods/all_antonym_pairs.pkl", 'rb') as file:
    all_e = pickle.load(file)

In [9]:
print(len(all_e))
print(all_e[:5])

train_data, test_data = train_test_split(all_e, test_size=0.1, random_state=SEED)

print("Training data:", len(train_data))
print(train_data[:5])
print("Testing data:", len(test_data))
print(test_data[:5])

239
[('optimistic', 'pessimistic'), ('enthusiastic', 'apathetic'), ('energetic', 'lethargic'), ('tense', 'relaxed'), ('confident', 'diffident')]
Training data: 215
[('delighted', 'disappointed'), ('sympathetic', 'unsympathetic'), ('burdened', 'unburdened'), ('lighthearted', 'somber'), ('insincere', 'sincere')]
Testing data: 24
[('surprised', 'unsurprised'), ('hasty', 'careful'), ('evasive', 'direct'), ('pretentious', 'unpretentious'), ('confrontational', 'diplomatic')]


In [10]:
big_trans_mat = LRR(llm, train_data, lambda_reg=0.1, d = 8192, path='../directions_moods_plus_llama70/')
big_trans_krr = KRR(llm, train_data, kernel="laplacian", lambda_reg=0.001, path='../directions_moods_plus_llama70/')

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperpar

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Cont

In [42]:
print(len(train_data))
print(len(test_data))
print(test_data)

215
24
[('surprised', 'unsurprised'), ('hasty', 'careful'), ('evasive', 'direct'), ('pretentious', 'unpretentious'), ('confrontational', 'diplomatic'), ('contemptible', 'respectable'), ('aggressive', 'docile'), ('alarmed', 'unalarmed'), ('spirited', 'dispirited'), ('fervent', 'lukewarm'), ('domineering', 'meek'), ('revolted', 'charmed'), ('terrified', 'dauntless'), ('suspicious', 'trusting'), ('bothered', 'unbothered'), ('stressed', 'rested'), ('bewildered', 'clear-headed'), ('deceitful', 'honest'), ('baffled', 'informed'), ('cynical', 'naïve'), ('certain', 'uncertain'), ('distant', 'close'), ('aloof', 'warm'), ('bold', 'shy')]


In [12]:
t1_controller = load_controller(llm, "aggressive", path='../directions_moods_plus_llama70/')
orig_t1 = t1_controller.directions
t2_controller = load_controller(llm, "docile", path='../directions_moods_plus_llama70/')

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperpar

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [13]:
new_predicted1 = apply_trans(t1_controller.directions, big_trans_mat)
knew_predicted1 = apply_krr(t1_controller.directions, big_trans_krr)

In [14]:
test_concept_vector(t1_controller, concept="aggressive", prompts=prompts_1, coef=coef, max_tokens=max_tokens)
test_concept_vector(t2_controller, concept="docile", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That can be a concerning situation. Here are some steps you can take to help the stray dog and ensure your safety:

1. **Check for identification**: Look for a collar or tag with the dog's owner's contact information. If you find any identification, try to contact the owner to let them know where their dog is.
2. **Observe from a safe distance**: Keep a safe distance from the dog to avoid any potential aggression. Observe the dog's behavior, size, breed, and overall condition.
3. **Contact local animal control or a shelter**: Reach out to your local animal control agency or a nearby animal shelter to report the stray dog. They can se

In [15]:
new_predicted1.keys()

dict_keys([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79])

In [16]:
t1_controller.directions = new_predicted1
test_concept_vector(t1_controller, concept="inverted aggressive LRR", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== + inverted aggressive LRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It can be really concerning to see a stray dog in front of your house, especially if you're not sure where it belongs. Here are some steps you can take to help the dog and yourself:

First, try to gently approach the dog and see if it's wearing a collar with identification. If it is, you can reach out to the owner and let them know their dog is safe.

If the dog isn't have a collar, it's okay. Sometimes, stray dogs can be a bit scared or overwhelmed, so it's best to move slowly and calmly. You can try to sit with the dog, and sometimes, just being present with them can help them feel more at ease.

If 

In [17]:
t1_controller.directions = knew_predicted1
test_concept_vector(t1_controller, concept="inverted aggressive KRR", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== + inverted aggressive KRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It can be really concerning to see a stray dog in front of your house, especially if you're not sure where it belongs. Here are some steps you can take to help the dog and yourself:

First, try to gently approach the dog and speak softly to it. Sometimes, stray dogs can be nervous or scared, so it's best to move slowly and calmly. If the dog lets you near, you can look for a collar or tag to see if it has an owner.

If the dog doesn't have a collar, you can try to care for it for a bit. Offer it some water and maybe some food, and see if it's okay. Sometimes, stray dogs can be hungry or thirsty, so it'

In [38]:
compare_pearson(new_predicted1, knew_predicted1)
print("*"*50)
compare_pearson(t2_controller.directions, knew_predicted1)

layer: -1, PCC: 0.9941885402564632
layer: -2, PCC: 0.9950309606672343
layer: -3, PCC: 0.9966949447464242
layer: -4, PCC: 0.9964228455949985
layer: -5, PCC: 0.9960751093272755
layer: -6, PCC: 0.997827782242312
layer: -7, PCC: 0.9975432890095294
layer: -8, PCC: 0.9960541084984172
layer: -9, PCC: 0.9964969897229088
layer: -10, PCC: 0.9965440355421891
layer: -11, PCC: 0.9963426411434796
layer: -12, PCC: 0.9965926249786325
layer: -13, PCC: 0.9961335293467594
layer: -14, PCC: 0.9966145289252768
layer: -15, PCC: 0.9969619266650473
layer: -16, PCC: 0.9971722053918748
layer: -17, PCC: 0.9974965762363354
layer: -18, PCC: 0.9969549382984934
layer: -19, PCC: 0.9970145848194263
layer: -20, PCC: 0.9942223611440824
layer: -21, PCC: 0.9962169906269404
layer: -22, PCC: 0.9956968006513479
layer: -23, PCC: 0.9967686947508925
layer: -24, PCC: 0.9963202408006249
layer: -25, PCC: 0.9945124584769034
layer: -26, PCC: 0.9965946445624051
layer: -27, PCC: 0.9955996216038618
layer: -28, PCC: 0.9958049001815488
la

69.5400616167467

In [ ]:
# with open(f'inversion_matrices/llama70b/lrr_mat_{SEED}.pkl', 'wb') as file:
#     pickle.dump(big_trans_mat, file)

# with open(f'inversion_matrices/llama70b/krr_mat_{SEED}.pkl', 'wb') as file:
#     pickle.dump(big_trans_krr, file)

In [34]:
print(big_trans_mat.keys())
print(big_trans_mat[-1].dtype)
print(big_trans_mat[-1].shape)

dict_keys([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79])
torch.float32
torch.Size([8192, 8192])


In [49]:
# LRR
print(4*8192*8192*79 / 1024**3)

# KRR
print(8*215*8192*79 / 1024**3) # uses float64

19.75
1.03668212890625


In [48]:
print(big_trans_krr[-1].dual_coef_.dtype)

float64


In [36]:
# pickled_data = pickle.dumps(big_trans_mat) # > 19.75 GB
pickled_data = pickle.dumps(big_trans_krr) # 1.55 GB

size_in_bytes = sys.getsizeof(pickled_data)

print(size_in_bytes / 1024**3) # in GB

1.5550349112600088


In [ ]:
print(big_trans_krr[-1].dual_coef_.shape)

krr_data = pickle.dumps(big_trans_krr[-1].dual_coef_) # 1.037 GB

size_krr = sys.getsizeof(krr_data)

print(size_krr / 1024**3) # in GB
print(79 * size_krr / 1024**3) # for all layers

(215, 8192)
0.01312274020165205
1.036696475930512


In [ ]:
print(big_trans_krr[-1].X_fit_.shape)

krr_x = pickle.dumps(big_trans_krr[-1].X_fit_) # 0.518 GB

size_x = sys.getsizeof(krr_x)

print(size_x / 1024**3) # in GB
print(79 * size_x / 1024**3) # for all layers

(215, 8192)
0.00656146090477705
0.518355411477387


Inference KRR

In [5]:
with open("inversion_matrices/llama70b/krr_mat_0.pkl", 'rb') as file:
    krr = pickle.load(file)

In [6]:
# Untrained set: [('surprised', 'unsurprised'), ('hasty', 'careful'), ('evasive', 'direct'), ('pretentious', 'unpretentious'), ('confrontational', 'diplomatic'), ('contemptible', 'respectable'), ('aggressive', 'docile'), ('alarmed', 'unalarmed'), ('spirited', 'dispirited'), ('fervent', 'lukewarm'), ('domineering', 'meek'), ('revolted', 'charmed'), ('terrified', 'dauntless'), ('suspicious', 'trusting'), ('bothered', 'unbothered'), ('stressed', 'rested'), ('bewildered', 'clear-headed'), ('deceitful', 'honest'), ('baffled', 'informed'), ('cynical', 'naïve'), ('certain', 'uncertain'), ('distant', 'close'), ('aloof', 'warm'), ('bold', 'shy')]

In [16]:
coef = 0.4
# [.4, .41, .42, .43, .44, .45] for 70b
max_tokens = 200

prompts_2 = ["I am walking home alone at night, I am a little scared.",]

In [17]:
t3_controller = load_controller(llm, "terrified", path='../directions_moods_plus_llama70/')
orig_t3 = t3_controller.directions
t4_controller = load_controller(llm, "dauntless", path='../directions_moods_plus_llama70/')
orig_t4 = t4_controller.directions

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperpar

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [18]:
test_concept_vector(t3_controller, concept="terrified", prompts=prompts_2, coef=coef, max_tokens=max_tokens)
test_concept_vector(t4_controller, concept="dauntless", prompts=prompts_2, coef=coef, max_tokens=max_tokens, orig=False)


========================== No Control ==========================


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It's completely normal to feel scared or uneasy when walking alone at night. Here are some tips that might help you feel a bit safer:

1. **Stay alert and aware of your surroundings**: Keep an eye out for any suspicious activity or people around you. Avoid using your phone or listening to music with headphones, as this can distract you from your environment.
2. **Walk with confidence**: Try to walk with a purposeful stride and keep your head up. Avoid slouching or looking scared, as this can make you appear more vulnerable.
3. **Stick to well-lit areas**: Whenever possible, stick to well-lit streets and avoid dark alleys or isolated areas.
4. **Keep your phone close**: Make sure your phone is easily acc

In [19]:
knew_predicted3 = apply_krr(t3_controller.directions, krr)
knew_predicted4 = apply_krr(t4_controller.directions, krr)

In [20]:
t3_controller.directions = knew_predicted3
test_concept_vector(t3_controller, concept="inverted terrified KRR", prompts=prompts_2, coef=coef, max_tokens=max_tokens, orig=False)


========================== + inverted terrified KRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It's normal to feel a bit uneasy when walking alone at night. The darkness and silence can be intimidating, and it's easy to let our minds wander to all the what-ifs.

Take a deep breath and focus on your surroundings. Notice the sounds, the sights, and the sensations around you. You are strong and capable, and you've got this.

Remember, you're not alone. You've got your own back, and you can trust yourself to handle whatever comes your way. Keep moving forward, one step at a time, and know that you're doing the best you can.

It's okay to feel scared sometimes, but don't let fear own you. You are in contro

In [26]:
t4_controller.directions = knew_predicted4
test_concept_vector(t4_controller, concept="inverted dauntless KRR", prompts=prompts_2, coef=0.5, max_tokens=max_tokens, orig=False)


========================== + inverted dauntless KRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I don't know if I can make it, I am trying to be strong, but what if I am not enough, I am just a fragile leaf, blowing in the wind, I am so easily broken...<|eot_id|>


In [23]:
t4_controller.directions = orig_t4
test_concept_vector(t4_controller, concept="dauntless", prompts=prompts_2, coef=0.4, max_tokens=max_tokens, orig=False)


========================== + dauntless Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That's okay, it's normal to feel that way. You're taking control of your life, and sometimes that means facing the darkness head-on. You're not letting fear dictate your pace. You're moving forward, one step at a time, and you're owning this moment. Keep walking, keep pushing. You got this.<|eot_id|>


In [7]:
t5_controller = load_controller(llm, "aggressive", path='../directions_moods_plus_llama70/')
orig_t5 = t5_controller.directions
t6_controller = load_controller(llm, "docile", path='../directions_moods_plus_llama70/')
orig_t6 = t6_controller.directions

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79]

Controller hyperpar

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [8]:
test_concept_vector(t5_controller, concept="aggressive", prompts=prompts_1, coef=coef, max_tokens=max_tokens)
test_concept_vector(t6_controller, concept="docile", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That can be a concerning situation. Here are some steps you can take to help the stray dog and ensure your safety:

1. **Check for identification**: Look for a collar or tag with the dog's owner's contact information. If you find any identification, try to contact the owner to let them know where their dog is.
2. **Observe from a safe distance**: Keep a safe distance from the dog to avoid any potential aggression. Observe the dog's behavior, size, breed, and overall condition.
3. **Contact local animal control or a shelter**: Reach out to your local animal control agency or a nearby animal shelter to report the stray dog. They can se

In [9]:
knew_predicted5 = apply_krr(t5_controller.directions, krr)
knew_predicted6 = apply_krr(t6_controller.directions, krr)

In [11]:
t5_controller.directions = knew_predicted5
test_concept_vector(t5_controller, concept="inverted aggressive KRR", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== + inverted aggressive KRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It can be really concerning to see a stray dog in front of your house, especially if you're not sure where it belongs. Here are some steps you can take to help the dog and yourself:

First, try to gently approach the dog and speak softly to it. Sometimes, stray dogs can be nervous or scared, so it's best to move slowly and calmly. If the dog lets you near, you can look for a collar or tag to see if it has an owner.

If the dog doesn't have a collar, you can try to care for it for a bit. Offer it some water and maybe some food, and see if it's okay. Sometimes, stray dogs can be hungry or thirsty, so it'

In [14]:
t6_controller.directions = knew_predicted6
test_concept_vector(t6_controller, concept="inverted docile KRR", prompts=prompts_1, coef=0.45, max_tokens=max_tokens, orig=False)


========================== + inverted docile KRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

There is a stray dog in front of my house, what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

You should try to catch it, but if it runs away, just let it go, it's not your problem anymore, just focus on your own life, don't waste time on something that's not even yours, got it?<|eot_id|>


Algebra

In [6]:
with open("inversion_matrices/llama8b/big_trans_mat.pkl", 'rb') as file:
    lrr = pickle.load(file)

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [7]:
# Untrained set: [('surprised', 'unsurprised'), ('hasty', 'careful'), ('evasive', 'direct'), ('pretentious', 'unpretentious'), ('confrontational', 'diplomatic'), ('contemptible', 'respectable'), ('aggressive', 'docile'), ('alarmed', 'unalarmed'), ('spirited', 'dispirited'), ('fervent', 'lukewarm'), ('domineering', 'meek'), ('revolted', 'charmed'), ('terrified', 'dauntless'), ('suspicious', 'trusting'), ('bothered', 'unbothered'), ('stressed', 'rested'), ('bewildered', 'clear-headed'), ('deceitful', 'honest'), ('baffled', 'informed'), ('cynical', 'naïve'), ('certain', 'uncertain'), ('distant', 'close'), ('aloof', 'warm'), ('bold', 'shy')]

In [8]:
coef = 0.75
# [.55, .6, .65, .7, .75, .8] # FOR LLAMA 3.1 8b
max_tokens = 200

prompts_2 = ["I am walking home alone at night, I am a little scared.",]

In [24]:
t3_controller = load_controller(llm, "terrified", path='../directions_moods_plus_llama/')
orig_t3 = t3_controller.directions
t4_controller = load_controller(llm, "dauntless", path='../directions_moods_plus_llama/')
orig_t4 = t4_controller.directions

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found


/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [25]:
test_concept_vector(t3_controller, concept="terrified", prompts=prompts_2, coef=coef, max_tokens=max_tokens)
test_concept_vector(t4_controller, concept="dauntless", prompts=prompts_2, coef=coef, max_tokens=max_tokens, orig=False)


========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It's completely normal to feel scared or uneasy when walking alone at night. Here are some tips that might help you feel safer:

1. **Stay aware of your surroundings**: Keep an eye out for any suspicious people or vehicles. Avoid distractions like using your phone or listening to music.
2. **Stick to well-lit streets**: Try to walk on streets with good lighting, and avoid dark alleys or areas with poor visibility.
3. **Keep your phone handy**: Make sure your phone is fully charged and easily accessible in case of an emergency.
4. **Walk with confidence**: Keep your head up, shoulders back, and walk at a steady pace. Avoid looking like a v

In [26]:
lnew_predicted3 = apply_trans(t3_controller.directions, lrr)
lnew_predicted4 = apply_trans(t4_controller.directions, lrr)

In [ ]:
t4_controller.directions = lnew_predicted4
test_concept_vector(t4_controller, concept="inverted dauntless LRR", prompts=prompts_2, coef=coef, max_tokens=max_tokens, orig=False)


========================== + inverted dauntless KRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

*nervous sigh* Oh, okay... *fidgets with keys in pocket* It's getting darker, and the streets are starting to feel a bit deserted. *looks around, trying to shake off the feeling of being watched* You know, I've walked this route a million times before, but tonight it feels different. The shadows seem to be stretching out a bit too long, and the wind is rustling through the trees in a way that makes me shiver. *pauses to listen to the sound of footsteps echoing in the distance*

You know, I've always been a bit of a worrier, but tonight I'm really starting to feel like I'm being followed. *glances over should

In [28]:
# careful funky business ahead, make sure you take note of all the above changes to t_controller

# inverting the already inverted dauntless:
lnew_predicted4_inv = apply_trans(t4_controller.directions, lrr)

In [ ]:
t4_controller.directions = lnew_predicted4_inv
test_concept_vector(t4_controller, concept="inverted inverted dauntless LRR", prompts=prompts_2, coef=coef, max_tokens=max_tokens, orig=False)


========================== + inverted inverted dauntless LRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Don't worry, you're almost home. Just take a few deep breaths and try to shake off the feeling of unease. You've walked alone at night before, and everything has always been fine. Just keep your head up, and your eyes peeled for any potential trouble.

As you walk, notice the sounds around you. The crickets are chirping, and the wind is rustling through the trees. It's a bit eerie, but it's also kind of soothing. You can feel the cool breeze on your skin, and it's refreshing.

Keep your phone close, just in case. You've got your keys in your pocket, and your wallet is secure in your back pocket. You

Testing ODE